<a href="https://colab.research.google.com/github/sriksmachi/ishara/blob/main/gesture_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
# !pip install scipy
# !pip install Pillow
# !pip install imageio
# !pip install tensorflow
# !pip install matplotlib
#!pip install opencv-python
# !pip install gdown

In [ ]:
import numpy as np
import os
import imageio
import datetime
import os
import cv2
import pathlib
import gdown
from scipy import misc

We set the random seed so that the results don't vary drastically.

In [ ]:
import random as rn
from keras import backend as K
import tensorflow as tf

rn.seed(30)
np.random.seed(30)
tf.compat.v1.random.set_random_seed(30)

In [ ]:
print(tf.__version__)

2.4.1


In [ ]:
!python --version

Python 3.7.10


In [ ]:
!nvidia-smi

Mon Apr 26 10:31:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Data

In [ ]:
!gdown 'https://drive.google.com/uc?export=download&id=1kl5iauN3iVCPnSHys0xvTUnAd41CNu3S'

Downloading...
From: https://drive.google.com/uc?export=download&id=1kl5iauN3iVCPnSHys0xvTUnAd41CNu3S
To: /content/Project_data.zip
740MB [00:05, 158MB/s]Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/gdown/cli.py", line 61, in main
    quiet=args.quiet,
  File "/usr/local/lib/python2.7/dist-packages/gdown/download.py", line 101, in download
    for chunk in res.iter_content(chunk_size=CHUNK_SIZE):

746MB [00:05, 140MB/s]


In [ ]:
!unzip /content/Project_data.zip

Archive:  /content/Project_data.zip
replace Project_data/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
## Use this to load the data if the data exists on gdrive and using google colab for 

data_dir="Project_data/"
data_dir_train = "Project_data/train/"
data_dir_val = 'Project_data/val/'

train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())


In [ ]:
# ## Use this to load the data if the data exists on Azure ML
# # azureml-core of version 1.0.72 or higher is required
# from azureml.core import Workspace, Dataset

# subscription_id = '925a5ad8-a21a-48c0-92a2-5a8a2a4dfc46'
# resource_group = 'machinelearning-workbench'
# workspace_name = 'sriks-azureml'

# workspace = Workspace(subscription_id, resource_group, workspace_name)

# dataset = Dataset.get_by_name(workspace, name='gesture_ds')

# ## Uncommenting the below will initiate the download, set Overwrite=True if overwriting existing files is intended. 
# # doc: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.filedataset?view=azure-ml-py#download-target-path-none--overwrite-false-
# dataset.download(target_path='Project_data/', overwrite=False)

# train_path = 'Project_data/train'
# val_path = 'Project_data/val'

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:

train_doc = np.random.permutation(open(data_dir+'train.csv').readlines())
val_doc = np.random.permutation(open(data_dir+'val.csv').readlines())

## Visualization

Let us plot some sample images and run some transformations on the image so see the impact.

In [ ]:
# # ## Pick some random sequence
# import matplotlib.pyplot as plt


# random_sequence = train_doc[np.random.randint(len(train_doc))].strip().split(';')[0]+'/'
# images = os.listdir(data_dir_train + random_sequence)
# # Create a code to visualize one instance of all the 30 images present in the sequence
# plt.figure(figsize=(10, 10))
# i = 0
# random_images = []
# for img in images:
#   ax = plt.subplot(6, 5, i + 1)
#   i = i + 1
#   random_images.append(os.path.join(data_dir_train, random_sequence, img))
#   img_bgr = cv2.imread(os.path.join(data_dir_train, random_sequence, img))
#   img_mp = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB) # This is needed because opencv uses BGR convention and matplotlib uses RGB
#   imgplot= plt.imshow(img_mp)
#   plt.axis("off")
#   plt.plot()

In [ ]:
# # Random Image - Original
# random_image = random_images[np.random.randint(30)]
# image_bgr = cv2.imread(random_image)
# plt.imshow(cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))
# print(image_bgr.shape)

In [ ]:
# ## Cropped Image Sample
# crop_img = image_bgr[10:120, 10:160]
# plt.imshow(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB))
# print(crop_img.shape)

In [ ]:
# # Resize Image Sample
# dim = (120, 120)
# resized_img = cv2.resize(crop_img, dim, interpolation = cv2.INTER_AREA)
# plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
# print(resized_img.shape)

In [ ]:
# # Normalized Image
# plt.imshow(cv2.cvtColor(cv2.normalize(resized_img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F), cv2.COLOR_BGR2RGB))

## Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def crop_image(image, size = 10):
     # cropping the image
    image_x = image.shape[0]
    image_y = image.shape[1]
    return image[size:image_x, size:image_y]

def normalize_minmax(image_resized, batch_data, folder, idx):
    batch_data[folder,idx,:,:,0] = cv2.normalize(image_resized[:,:,0], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    batch_data[folder,idx,:,:,1] = cv2.normalize(image_resized[:,:,1], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    batch_data[folder,idx,:,:,2] = cv2.normalize(image_resized[:,:,2], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    return batch_data   

def crop_and_normalize(image,batch_data, folder, idx, y=120,z=120):
    
    #crop the images and resize them. Note that the images are of 2 different shape 
    #and the conv3D will throw error if the inputs in a batch have different shapes
    image_cropped = crop_image(image)
    image_resized = cv2.resize(image_cropped, (z,y), interpolation = cv2.INTER_AREA)
                    
    # using min max normalization.
    pending_batch_data = normalize_minmax(image_resized, batch_data, folder, idx)
    return pending_batch_data


def create_batch_data(t, source_path, img_idx, folder, batch, batch_size, batch_data, batch_labels,dim=(120,120)):
    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    
        image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+
                           imgs[item]).astype(np.float32)
                    
        # using min max normalization.
        batch_data = crop_and_normalize(image, batch_data, folder, idx,y=dim[0],z=dim[1])
                    
    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
    return batch_data, batch_labels

def generator(source_path, folder_list, dim=(120,120), batch_size=30, ablation=None,samples=[5,10,15]):
    print( 'Source path = ', source_path, '; batch size =', batch_size, 'dim=', dim)
    x = len(samples) # number of taken from each video
    img_idx = samples #create a list of image numbers you want to use for a particular video
    y = dim[0] # image dim
    z = dim[1] # image dim
    while True:
        if ablation is not None: 
            t = np.random.permutation(folder_list[:ablation])
        else:
            t = np.random.permutation(folder_list)
        num_batches =  len(t) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                
                ## get images from the folder
                batch_data_labels = create_batch_data(t, source_path, img_idx, folder, batch, batch_size, batch_data, batch_labels,dim)
                batch_data = batch_data_labels[0]
                batch_labels = batch_data_labels[1]
                
            yield batch_data, batch_labels 
            
        # The length of the folder list could leave some residue folders, the below code deals with it
        # Number of pending batches
        pending_batches = len(t) % batch_size
        
        pending_batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        pending_batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
        
        for folder in range(pending_batches): # iterate over the batch_size

            # getimages from the folder
            pending_batch_data_labels = create_batch_data(t, source_path, img_idx, folder, num_batches, batch_size, pending_batch_data, pending_batch_labels)
            pending_batch_data = pending_batch_data_labels[0]
            pending_batch_labels = pending_batch_data_labels[1]       
            
            
        yield pending_batch_data, pending_batch_labels
            

# g = generator(data_dir_train, train_doc)
# next(g)

In [ ]:
# g = generator(data_dir_train, train_doc, (120,120), 30, ablation=None, samples=30)
# p = 0
# for k in g:
#     p+=1
# print(p)
# # next(g)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model

#### Objective

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

### 3D Convolution

We will build the model using 3D Convolution Architecture

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
samples_per_image=list(range(0,30,2))

# Building a 3 D Convolution model.
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(len(samples_per_image),120,120,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(2, 2, 2), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))


model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser =  'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 13, 118, 118, 32)  2624      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 6, 59, 59, 32)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 5, 58, 58, 64)     16448     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 2, 29, 29, 64)     0         
_________________________________________________________________
flatten (Flatten)            (None, 107648)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               27558144  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
# Reusable function to run 3d convolution
def run_3d_convolution(num_epochs, batch_size, dim=(120,120), ablation_size=None, callbacks_list=[]):
  if ablation_size is None:
    num_train_sequences = len(train_doc)
    num_val_sequences = len(val_doc)
  else:
    num_train_sequences = len(train_doc[:ablation_size])
    num_val_sequences = len(val_doc[:ablation_size])

  print('# training sequences =', num_train_sequences)
  print('# validation sequences =', num_val_sequences)
  print ('# epochs =', num_epochs)

  if (num_train_sequences%batch_size) == 0:
     steps_per_epoch = int(num_train_sequences/batch_size)
  else:
     steps_per_epoch = (num_train_sequences//batch_size) + 1

  if (num_val_sequences%batch_size) == 0:
      validation_steps = int(num_val_sequences/batch_size)
  else:
      validation_steps = (num_val_sequences//batch_size) + 1

  print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

  train_generator = generator(data_dir_train, train_doc, dim=dim, batch_size=batch_size, ablation=ablation_size, samples=samples_per_image)
  val_generator = generator(data_dir_val, val_doc, dim=dim, batch_size=batch_size, ablation=ablation_size, samples=samples_per_image)

  print('='*100)

  model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

### Ablation Experiment

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`. 
We will fit the model with 1 epoch just to validate if the model is working.
Notice I have not added any callbacks yet, we will do it in the later sections

In [ ]:
# Experiment Parameters
ablation_size=100
num_epochs = 1
batch_size=30
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size)

# training sequences = 100
# validation sequences = 100
# epochs = 1
steps_per_epoch: 4, validation_steps: 4
Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
4/4 [==============================] - 43s 3s/step - loss: 5.1041 - categorical_accuracy: 0.2644 - val_loss: 1.6216 - val_categorical_accuracy: 0.1917


The model seems to be working well, there are no errors. 

### Overfitting on training data. 

Let us overfit on the training data to see if the model is able to learn from the data. We are going to use less data and run for more epochs and see if the model is able to improve the accuracy and reduce the loss.

In [ ]:
ablation_size=300
batch_size=10
num_epochs=10
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size)

# training sequences = 300
# validation sequences = 100
# epochs = 10
steps_per_epoch: 30, validation_steps: 10
Source path =  Project_data/train/ ; batch size = 10 dim= (120, 120)
Epoch 1/10
30/30 [==============================] - 15s 494ms/step - loss: 1.5853 - categorical_accuracy: 0.2767 - val_loss: 1.4875 - val_categorical_accuracy: 0.3600
Epoch 2/10
30/30 [==============================] - 13s 450ms/step - loss: 1.3208 - categorical_accuracy: 0.4400 - val_loss: 1.3518 - val_categorical_accuracy: 0.4300
Epoch 3/10
30/30 [==============================] - 13s 440ms/step - loss: 1.0519 - categorical_accuracy: 0.6000 - val_loss: 1.2726 - val_categorical_accuracy: 0.4900
Epoch 4/10
30/30 [==============================] - 13s 440ms/step - loss: 0.8156 - categorical_accuracy: 0.6800 - val_loss: 1.1767 - val_categorical_accuracy: 0.5100
Epoch 5/10
30/30 [==============================] - 13s 442ms/step - loss: 0.5478 - categorical_accuracy: 0.7933 - val_loss: 1.1730 - val_categorical_a

The training loss reduced and accuracy increased, the model is able to learn well. There is significant gap between the training and validation accuracy so we have successfully overfit the model.

### Callbacks

Here we define few callbacks which we will use later when we fit the complete model.

In [ ]:
curr_dt_time = datetime.datetime.now()
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)

ES = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

callbacks_list = [LR, ES]

### Final Experiment

In [ ]:
#Experiment Parameters
ablation_size=None
batch_size=30
num_epochs=20
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size,callbacks_list)

# training sequences = 663
# validation sequences = 100
# epochs = 20
steps_per_epoch: 23, validation_steps: 4
Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
Epoch 1/20
23/23 [==============================] - 28s 1s/step - loss: 0.9613 - categorical_accuracy: 0.6232 - val_loss: 0.8870 - val_categorical_accuracy: 0.4750
Epoch 2/20
23/23 [==============================] - 26s 1s/step - loss: 0.8479 - categorical_accuracy: 0.6754 - val_loss: 0.7713 - val_categorical_accuracy: 0.5750
Epoch 3/20
23/23 [==============================] - 25s 1s/step - loss: 0.7402 - categorical_accuracy: 0.6942 - val_loss: 0.7274 - val_categorical_accuracy: 0.5833
Epoch 4/20
23/23 [==============================] - 25s 1s/step - loss: 0.6314 - categorical_accuracy: 0.7478 - val_loss: 0.9686 - val_categorical_accuracy: 0.4500
Epoch 5/20
23/23 [==============================] - 25s 1s/step - loss: 0.5620 - categorical_accuracy: 0.7855 - val_loss: 0.7750 - val_categorical_accuracy: 0.5500


In [ ]:
ablation_size=None
batch_size=10
num_epochs=20
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size,callbacks_list)

# training sequences = 663
# validation sequences = 100
# epochs = 20
steps_per_epoch: 67, validation_steps: 10
Source path =  Project_data/train/ ; batch size = 10 dim= (120, 120)
Epoch 1/20
67/67 [==============================] - 25s 382ms/step - loss: 0.1071 - categorical_accuracy: 0.9806 - val_loss: 0.8996 - val_categorical_accuracy: 0.7000
Epoch 2/20
67/67 [==============================] - 25s 382ms/step - loss: 0.0960 - categorical_accuracy: 0.9866 - val_loss: 0.8741 - val_categorical_accuracy: 0.7100
Epoch 3/20
67/67 [==============================] - 26s 385ms/step - loss: 0.0849 - categorical_accuracy: 0.9896 - val_loss: 0.8924 - val_categorical_accuracy: 0.6900
Epoch 4/20
67/67 [==============================] - 26s 385ms/step - loss: 0.0778 - categorical_accuracy: 0.9881 - val_loss: 0.8832 - val_categorical_accuracy: 0.6800
Epoch 5/20
67/67 [==============================] - 25s 381ms/step - loss: 0.0695 - categorical_accuracy: 0.9896 - val_loss: 0.9115 - val_categorical_a

### Adding one more Conv Layer

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
samples_per_image=list(range(0,30,2))

# Building a 3 D Convolution model.
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(len(samples_per_image),120, 120,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, kernel_size=(2, 2, 2), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# learnings from https://machinelearningmastery.com/introduction-to-1x1-convolutions-to-reduce-the-complexity-of-convolutional-neural-networks/
model.add(Conv3D(128, kernel_size=(1, 1, 1), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))

optimiser =  'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 13, 118, 118, 32)  2624      
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 6, 59, 59, 32)     0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 5, 58, 58, 64)     16448     
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 2, 29, 29, 64)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 2, 29, 29, 128)    8320      
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 1, 14, 14, 128)    0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)            

### Overfitting the model intentionally.

In [ ]:
ablation_size=300
batch_size=30
num_epochs=10
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size)

# training sequences = 300
# validation sequences = 100
# epochs = 10
steps_per_epoch: 10, validation_steps: 4
Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
Epoch 1/10
10/10 [==============================] - 13s 1s/step - loss: 0.0108 - categorical_accuracy: 1.0000 - val_loss: 0.8441 - val_categorical_accuracy: 0.5167
Epoch 2/10
10/10 [==============================] - 14s 2s/step - loss: 0.0099 - categorical_accuracy: 0.9000 - val_loss: 0.8371 - val_categorical_accuracy: 0.5167
Epoch 3/10
10/10 [==============================] - 13s 1s/step - loss: 0.0086 - categorical_accuracy: 0.9000 - val_loss: 0.8518 - val_categorical_accuracy: 0.5250
Epoch 4/10
10/10 [==============================] - 12s 1s/step - loss: 0.0110 - categorical_accuracy: 0.9000 - val_loss: 0.7689 - val_categorical_accuracy: 0.5333
Epoch 5/10
10/10 [==============================] - 13s 1s/step - loss: 0.0089 - categorical_accuracy: 0.9000 - val_loss: 0.8530 - val_categorical_accuracy: 0.5167


Training accuracy is around 90, and there is significant gap between training and validation accuracy, we have successfully overfit the model 

In [ ]:
#Experiment Parameters
ablation_size=None
batch_size=10
num_epochs=30
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size,callbacks_list)

# training sequences = 663
# validation sequences = 100
# epochs = 30
steps_per_epoch: 67, validation_steps: 10
Source path =  Project_data/train/ ; batch size = 10 dim= (120, 120)
Epoch 1/30
67/67 [==============================] - 26s 382ms/step - loss: 2.2399 - categorical_accuracy: 0.2527 - val_loss: 1.5012 - val_categorical_accuracy: 0.1900
Epoch 2/30
67/67 [==============================] - 25s 377ms/step - loss: 1.4672 - categorical_accuracy: 0.3392 - val_loss: 1.3248 - val_categorical_accuracy: 0.4900
Epoch 3/30
67/67 [==============================] - 25s 381ms/step - loss: 1.2450 - categorical_accuracy: 0.5313 - val_loss: 1.2403 - val_categorical_accuracy: 0.4900
Epoch 4/30
67/67 [==============================] - 25s 376ms/step - loss: 1.1073 - categorical_accuracy: 0.5933 - val_loss: 1.2571 - val_categorical_accuracy: 0.5100
Epoch 5/30
67/67 [==============================] - 25s 379ms/step - loss: 0.9722 - categorical_accuracy: 0.6020 - val_loss: 1.2188 - val_categorical_a

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
samples_per_image=list(range(0,30,2))

# Building a 3 D Convolution model.
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(len(samples_per_image),120, 120,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization()),

model.add(Conv3D(64, kernel_size=(2, 2, 2), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization()),

# learnings from https://machinelearningmastery.com/introduction-to-1x1-convolutions-to-reduce-the-complexity-of-convolutional-neural-networks/
model.add(Conv3D(128, kernel_size=(1, 1, 1), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization()),

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))

optimiser =  'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_14 (Conv3D)           (None, 13, 118, 118, 32)  2624      
_________________________________________________________________
max_pooling3d_14 (MaxPooling (None, 6, 59, 59, 32)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 59, 59, 32)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 6, 59, 59, 32)     128       
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 5, 58, 58, 64)     16448     
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 2, 29, 29, 64)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 29, 29, 64)    

### Ablation Experiment

In [ ]:
# Experiment Parameters
ablation_size=100
num_epochs = 1
batch_size=30
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size)

# training sequences = 100
# validation sequences = 100
# epochs = 1
steps_per_epoch: 4, validation_steps: 4
Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
4/4 [==============================] - 9s 3s/step - loss: 2.5212 - categorical_accuracy: 0.1611 - val_loss: 1.5685 - val_categorical_accuracy: 0.2000


### Overfitting

In [ ]:
ablation_size=300
batch_size=10
num_epochs=10
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size)

# training sequences = 300
# validation sequences = 100
# epochs = 10
steps_per_epoch: 30, validation_steps: 10
Source path =  Project_data/train/ ; batch size = 10 dim= (120, 120)
Epoch 1/10
30/30 [==============================] - 13s 447ms/step - loss: 1.5504 - categorical_accuracy: 0.4367 - val_loss: 1.8248 - val_categorical_accuracy: 0.3700
Epoch 2/10
30/30 [==============================] - 13s 448ms/step - loss: 0.8286 - categorical_accuracy: 0.6767 - val_loss: 3.7559 - val_categorical_accuracy: 0.2200
Epoch 3/10
30/30 [==============================] - 13s 436ms/step - loss: 0.6084 - categorical_accuracy: 0.7567 - val_loss: 4.5915 - val_categorical_accuracy: 0.2100
Epoch 4/10
30/30 [==============================] - 13s 448ms/step - loss: 0.4099 - categorical_accuracy: 0.8233 - val_loss: 7.4317 - val_categorical_accuracy: 0.2100
Epoch 5/10
30/30 [==============================] - 13s 441ms/step - loss: 0.3720 - categorical_accuracy: 0.8433 - val_loss: 5.8541 - val_categorical_a

### Final Experiment

In [ ]:
#Experiment Parameters
ablation_size=None
batch_size=30
num_epochs=30
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size,callbacks_list)

# training sequences = 663
# validation sequences = 100
# epochs = 30
steps_per_epoch: 23, validation_steps: 4
Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
Epoch 1/30
23/23 [==============================] - 27s 1s/step - loss: 0.1553 - categorical_accuracy: 0.9174 - val_loss: 1.3541 - val_categorical_accuracy: 0.4417
Epoch 2/30
23/23 [==============================] - 29s 1s/step - loss: 0.1312 - categorical_accuracy: 0.9130 - val_loss: 1.4157 - val_categorical_accuracy: 0.6083
Epoch 3/30
23/23 [==============================] - 27s 1s/step - loss: 0.2113 - categorical_accuracy: 0.9217 - val_loss: 2.5982 - val_categorical_accuracy: 0.2917
Epoch 4/30
23/23 [==============================] - 25s 1s/step - loss: 0.1359 - categorical_accuracy: 0.9319 - val_loss: 1.1372 - val_categorical_accuracy: 0.4417
Epoch 5/30
23/23 [==============================] - 26s 1s/step - loss: 0.1794 - categorical_accuracy: 0.9130 - val_loss: 3.2580 - val_categorical_accuracy: 0.3083


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
samples_per_image=list(range(0,30,2))

# Building a 3 D Convolution model.
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(len(samples_per_image),120, 120,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25)),

model.add(Conv3D(64, kernel_size=(2, 2, 2), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25)),

# learnings from https://machinelearningmastery.com/introduction-to-1x1-convolutions-to-reduce-the-complexity-of-convolutional-neural-networks/
model.add(Conv3D(128, kernel_size=(1, 1, 1), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dropout(0.25)),
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))

optimiser =  'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 13, 118, 118, 32)  2624      
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 6, 59, 59, 32)     0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 6, 59, 59, 32)     0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 5, 58, 58, 64)     16448     
_________________________________________________________________
max_pooling3d_18 (MaxPooling (None, 2, 29, 29, 64)     0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 2, 29, 29, 64)     0         
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 2, 29, 29, 128)   

In [ ]:
#Experiment Parameters
ablation_size=None
batch_size=30
num_epochs=20
dim=(120,120)

run_3d_convolution(num_epochs, batch_size, dim, ablation_size,callbacks_list)

# training sequences = 663
# validation sequences = 100
# epochs = 20
steps_per_epoch: 23, validation_steps: 4
Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
Epoch 1/20
23/23 [==============================] - 28s 1s/step - loss: 4.7297 - categorical_accuracy: 0.1932 - val_loss: 1.3013 - val_categorical_accuracy: 0.2417
Epoch 2/20
23/23 [==============================] - 33s 2s/step - loss: 1.5556 - categorical_accuracy: 0.2547 - val_loss: 1.2635 - val_categorical_accuracy: 0.2083
Epoch 3/20
23/23 [==============================] - 30s 1s/step - loss: 1.5231 - categorical_accuracy: 0.2497 - val_loss: 1.2793 - val_categorical_accuracy: 0.1417
Epoch 4/20
23/23 [==============================] - 25s 1s/step - loss: 1.4919 - categorical_accuracy: 0.3101 - val_loss: 1.2280 - val_categorical_accuracy: 0.3083
Epoch 5/20
23/23 [==============================] - 26s 1s/step - loss: 1.4681 - categorical_accuracy: 0.3131 - val_loss: 1.2240 - val_categorical_accuracy: 0.3250


### CNN + RNN Model using Transfer Learning

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,TimeDistributed,GRU,Conv2D,MaxPooling2D
from tensorflow.keras.models import Sequential

In [ ]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(120,120,3))
for layer in resnet.layers[:-10]:
    layer.trainable=False

cnn = Sequential([resnet])
cnn.add(Conv2D(16, kernel_size=(3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(120,120,3)))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(Conv2D(32, kernel_size=(2, 2), activation='relu', kernel_initializer='he_uniform'))
# cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
cnn.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))

model = Sequential()
model.add(TimeDistributed(cnn,input_shape=(10,120,120,3)))
model.add(GRU(16,input_shape=(None,30,256),return_sequences=True))
model.add(GRU(8))
model.add(Dense(5,activation='softmax'))


94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 10, 256)           23958960  
_________________________________________________________________
gru (GRU)                    (None, 10, 16)            13152     
_________________________________________________________________
gru_1 (GRU)                  (None, 8)                 624       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 45        
Total params: 23,972,781
Trainable params: 4,850,733
Non-trainable params: 19,122,048
_________________________________________________________________


In [ ]:
optimiser =  'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 10, 256)           23958960  
_________________________________________________________________
gru (GRU)                    (None, 10, 16)            13152     
_________________________________________________________________
gru_1 (GRU)                  (None, 8)                 624       
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 45        
Total params: 23,972,781
Trainable params: 4,850,733
Non-trainable params: 19,122,048
_________________________________________________________________
None


### Ablation Experiment

In [ ]:
ablation_size=100
batch_size=30
num_epochs=1

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))

# training sequences = 663
# validation sequences = 100
# epochs = 1
steps_per_epoch: 23, validation_steps: 4


In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
23/23 [==============================] - 33s 973ms/step - loss: 1.3870 - accuracy: 0.2782 - val_loss: 1.4787 - val_accuracy: 0.3167


## Overfitting Intentionally

In [ ]:
ablation_size=None
batch_size=30
num_epochs=10

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))

# training sequences = 663
# validation sequences = 100
# epochs = 10
steps_per_epoch: 23, validation_steps: 4


In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
Epoch 1/10
23/23 [==============================] - 22s 956ms/step - loss: 1.5143 - accuracy: 0.2710 - val_loss: 1.3278 - val_accuracy: 0.1917
Epoch 2/10
23/23 [==============================] - 21s 929ms/step - loss: 1.4501 - accuracy: 0.3290 - val_loss: 1.3306 - val_accuracy: 0.2250
Epoch 3/10
23/23 [==============================] - 21s 931ms/step - loss: 1.4263 - accuracy: 0.3362 - val_loss: 1.3654 - val_accuracy: 0.1833
Epoch 4/10
23/23 [==============================] - 21s 914ms/step - loss: 1.3738 - accuracy: 0.3507 - val_loss: 1.3854 - val_accuracy: 0.1833
Epoch 5/10
23/23 [==============================] - 21s 915ms/step - loss: 1.3559 - accuracy: 0.3870 - val_loss: 1.3315 - val_accuracy: 0.1917
Epoch 6/10
23/23 [==============================] - 21s 932ms/step - loss: 1.3145 - accuracy: 0.3986 - val_loss: 1.2358 - val_accuracy: 0.2833
Epoch 7/10
23/23 [==============================] - 21s 927ms/step - loss

## Final Experiment

In [ ]:
ablation_size=None
batch_size=30
num_epochs=20

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
print ('# epochs =', num_epochs)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print('steps_per_epoch: {}, validation_steps: {}'.format(steps_per_epoch, validation_steps))

train_generator = generator(data_dir_train, train_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))
val_generator = generator(data_dir_val, val_doc, dim=(120,120), batch_size=batch_size, ablation=ablation_size, samples=list(range(0,30,3)))

# training sequences = 663
# validation sequences = 100
# epochs = 20
steps_per_epoch: 23, validation_steps: 4


In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train/ ; batch size = 30 dim= (120, 120)
Epoch 1/20
23/23 [==============================] - 21s 939ms/step - loss: 1.2837 - accuracy: 0.4058 - val_loss: 1.3758 - val_accuracy: 0.2250
Epoch 2/20
23/23 [==============================] - 21s 930ms/step - loss: 1.1789 - accuracy: 0.4725 - val_loss: 1.3594 - val_accuracy: 0.2167
Epoch 3/20
23/23 [==============================] - 21s 916ms/step - loss: 1.1700 - accuracy: 0.4812 - val_loss: 1.1822 - val_accuracy: 0.3333
Epoch 4/20
23/23 [==============================] - 21s 906ms/step - loss: 1.1766 - accuracy: 0.5362 - val_loss: 1.2202 - val_accuracy: 0.3167
Epoch 5/20
23/23 [==============================] - 21s 933ms/step - loss: 1.1436 - accuracy: 0.5594 - val_loss: 1.2143 - val_accuracy: 0.3583
Epoch 6/20
23/23 [==============================] - 20s 903ms/step - loss: 1.1149 - accuracy: 0.5551 - val_loss: 1.2266 - val_accuracy: 0.3000
Epoch 7/20
23/23 [==============================] - 21s 931ms/step - loss